<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/03_elementary_Japanese_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **日本語データの扱い方**

* 日本語は、言語の性質上、文が空白によって単語へ分たれていない。
 * 同じように、文が空白によって単語へ分たれていない言語として、他にどのような言語があるか？
* そのため、まず最初に文を単語へ分割する必要がある。
* この作業を**形態素解析**と言う。
 * 長い文字列としての言語データを、単語のような細かい単位へ分割することを、一般にtokenizationと言う。
 * tokenizationの単位は、単語のように意味的なまとまりをもつ単位とは限らない。
 *cf. [SentencePiece](https://arxiv.org/pdf/1808.06226.pdf) ... [この解説](https://qiita.com/taku910/items/7e52f1e58d0ea6e7859c)も参照。


* ここでは、spaCyを介してSudachiという形態素解析器を使う。
 * 形態素解析器としては、他には、MeCabやJUMANが有名。

* 注
 * "ContextualVersionConflict ..."というエラーが出たら、上部メニューの「ランタイム」から「ランタイムを再起動」をクリックして、ランタイムを再起動してみてください。

## 1. 日本語NLPのためのspaCyの準備

### spaCyを最新版にアップデート

In [ ]:
!pip install -U spacy

* バージョンを確認

In [ ]:
!python -m spacy validate

### spaCyの日本語の訓練済み統計モデルをダウンロードする
* 「sm」で終わるものは、小規模なモデル。
* 「md」で終わるものが、中規模のモデル。
* 「lg」で終わるものが、大規模なモデル。
* 詳細は、以下を参照。
 * https://explosion.ai/blog/spacy-v2-3
* ダウンロード時間短縮のため、ここでは小規模モデルを使う。

In [ ]:
!python -m spacy download ja_core_news_sm

### Sudachiによる形態素解析
* spaCyの日本語統計モデルをインストールすると、自動的にSudachiがインストールされる。
 * https://github.com/WorksApplications/Sudachi
* そこで、sudachipyコマンドを使って、コマンドラインで形態素解析を試してみる。

In [ ]:
!echo "すもももももももものうち" | sudachipy

## 2. spaCyによる日本語NLP

### 日本語統計モデルのロード

* ここでランタイムを再起動する。
* その後、以下を実行。

In [ ]:
from spacy.lang.ja import Japanese

# Load SudachiPy with split mode A (default)
nlp = Japanese()

# Load SudachiPy with split mode B
#cfg = {"split_mode": "B"}
#nlp = Japanese(meta={"tokenizer": {"config": cfg}})

* 文章を作成

In [ ]:
doc = nlp('これはやっぱり相当冗長な日本語の文じゃないかなと思っていたのですが。')
print(doc.text)

### spaCyからSudachiを利用
* 分かち書きされた単語そのもの以外の、品詞などの情報も表示させている。
 * tagに「形容動詞」は無く、その語幹が「形状詞」とされている。
 * 「じゃない」の「ない」は、tagでは「形容詞」だが、posでは「AUX」である。

In [ ]:
for token in doc:
  print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')

# 課題３

* （この課題は、宿題ではなく、いまここで作業しつつ解いてしまいます。）
* Wikipediaの複数の記事を、lemmaを半角スペースでつないだ、長い文字列へ変換する。
 * ここでは、コンピュータ科学の様々な分野の記事を題材として使う。
* scikit-learnのCountVectorizerやTfidfVectorizerを使って、各記事における単語の出現頻度からなる文書ベクトルを得る。
* 特徴ベクトルどうしの類似度を計算し、「人工知能」分野と最も似ている順に　３つの分野がどの分野かを求める。
 * 答えは自分の感覚でチェック。
 * 文書ベクトルを作る時に、単語の品詞を名詞に限定するなど、品詞の情報を使うことで結果を改善できるかどうかも、余裕があれば試行錯誤する。

## 課題の手順(1)

### Wikipediaのエントリをダウンロードして形態素解析を適用

* Wikipediaの「人工知能」エントリをダウンロードする。
 * https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD
* そして、そこに含まれる段落（__`<p>`__タグで囲まれた範囲）を列挙する。
* 各段落のテキストに形態素解析を適用する。
* 形態素解析で得られたlemmaを半角スペースでつないで、エントリ全体をひとつの長い文字列にする。
 * text、つまり、単語に分たれたそのままの文字列を半角スペースでつなぐのではない。
 * lemma（原型に戻したもの）を半角スペースでつなぐこと。
 * posやtagを見て、不要そうな単語を適当に削除してもよい。


In [ ]:
# 「人工知能」エントリをダウンロードしてparserのインスタンスを作る。

from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 段落のテキストを取得する。

lines = list()
for para in soup.find_all('p'):
  lines.append(para.text)

In [ ]:
lines

In [ ]:
# Sudachiで形態素解析し、分かち書き後のlemmaを取得する。

x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
tokens = list()
for line in lines[:10]:
  for token in nlp(line):
    pos = token.pos_
    if not pos in x_pos:
      print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')
      tokens.append(token.lemma_)

In [ ]:
# すべてのlemmaを半角スペースでつないで、長い文字列にする。

doc_AI = ' '.join(tokens)
print(doc_AI)

* sudachiの設定を変えて使ってみる
 * https://www.anlp.jp/proceedings/annual_meeting/2019/pdf_dir/P8-5.pdf

In [ ]:
# Load SudachiPy with split mode C
cfg = {"split_mode": "C"}
nlp = Japanese(meta={"tokenizer": {"config": cfg}})

In [ ]:
# Sudachiで形態素解析し、分かち書き後のlemmaを取得する。

x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
tokens = list()
for line in lines[:10]:
  for token in nlp(line):
    pos = token.pos_
    if pos not in x_pos:
      print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')
      tokens.append(token.lemma_)

In [ ]:
# すべてのlemmaを半角スペースでつないで、長い文字列にする。

doc_AI = ' '.join(tokens)
print(doc_AI)

* 上記の操作をまとめておこなう関数を定義しておく。
 * 後で、各エントリについて、この関数を呼び出す。

In [ ]:
def morph(soup, nlp):

  lines = list()
  for para in soup.find_all('p'):
    lines.append(para.text)

  x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
  tokens = list()
  for line in lines:
    for token in nlp(line):
      pos = token.pos_
      if pos not in x_pos:
        tokens.append(token.lemma_)

  return ' '.join(tokens)

## 課題の手順(2)

### 「コンピュータ科学」の様々な分野に関するWikipediaエントリについて上記の作業を実行
*  「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。
* ただし、__`/wiki/`__という文字列で始まっているURLであり、かつ、テンプレートの状態でないものだけを抜き出す。
 * Wikipediaのクローリングについてもっと詳しく知りたい場合は、下記のページ等を参照されたい。
   * https://medium.com/@robinlphood/tutorial-a-simple-crawler-for-wikipedia-d7b6f6f55d5

In [ ]:
# 「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。

target_str = '表話編歴コンピュータ科学'
prefix = '/wiki/'

urls = dict()
for table in soup.find_all('table'):
  if table.text[:len(target_str)] != target_str: continue
  for td in table.find_all('td'):
    for a in td.find_all('a'):
      if not a.text: continue
      try:
        if a.text.find('英語版') == -1:
          href = a['href']
          if href[:len(prefix)] == prefix and href.find('/Template:') == -1 and href.find('/Category:') == -1:
            urls[a.text] = 'https://ja.wikipedia.org' + href
      except:
        continue

for k in urls:
  print(k, urls[k])

In [ ]:
# 各エントリをダウンロードしてparserのインスタンスを作り、辞書として保存。

soups = dict()
for k in urls:
  html = urlopen(urls[k]) 
  soups[k] = BeautifulSoup(html, 'html.parser')

In [ ]:
# 先ほど定義した関数morph()を使って各エントリを形態素解析し、lemmaが半角スペースで区切られた文字列へ変換する。

genre = list()
corpus = list()
for k in soups:
  genre.append(k)
  doc = morph(soups[k], nlp)
  corpus.append(doc)

In [ ]:
# 再利用するために、csvファイルとして保存しておく。

import pandas as pd

genre.append('人工知能')
corpus.append(doc_AI)

df = pd.DataFrame(list(zip(genre, corpus)), columns=['genre', 'text'])
print(df.head())
df.to_csv('cs_corpus.csv')

## 課題の手順(3)

### scikit-learnのCountVectorizerで単語の出現頻度を要素とするベクトルを作成
* これにより、各文書のベクトル表現が得られる。
* 興味のある対象のベクトル表現を得ることは、その対象を機械学習アルゴリズムの入力データとして使うための第一歩。

In [ ]:
# 先ほど作成したcsvファイルを読んで、textカラムをCountVectorizerのインスタンスでベクトル化する。
# ベクトルの要素は、各Wikipediaエントリにおける、各単語の出現回数になる。

from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('cs_corpus.csv')

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])

In [ ]:
# CountVectorizerで得られるのはsparse matrixである。
print(X)

In [ ]:
# sparse matrixから、通常のNumPyの配列へ変換する。
X = X.toarray()
print('文書数:{}; 語彙サイズ：{}'.format(*X.shape))

In [ ]:
# CountVectorizerによって作られた語彙を取得する。
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
# コーパス全体での出現頻度順で上位20単語を見てみる。

import numpy as np

vocab = np.array(vocab)
print(vocab[np.argsort(- X.sum(axis=0))][:20])

In [ ]:
# 「人工知能」エントリと他のエントリとの距離を求める。

from scipy.spatial import distance

genre = df['genre'].values.tolist()
index_AI = genre.index('人工知能')
print(f'「{genre[index_AI]}」と「{genre[10]}」との間での・・・')
print(f'ユークリッド距離: {np.linalg.norm(X[0] - X[index_AI])}')
print(f'内積: {np.dot(X[0], X[index_AI])}')
print(f'コサイン類似度: {distance.cosine(X[0], X[index_AI])}')

### ここから各自作業。
* 「人工知能」と、Wikipediaのエントリの類似度の上で、最も近いジャンルは、どれ？
* ユークリッド距離とコサイン類似度のうち、どちらを類似度として使った場合の答えが、より納得できる答えになっているか？

In [ ]:
genre